In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import geopandas as gpd

In [3]:
ds = pd.read_csv("California_Fire_Incidents.csv")
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1636 entries, 0 to 1635
Data columns (total 40 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   AcresBurned           1633 non-null   float64
 1   Active                1636 non-null   bool   
 2   AdminUnit             1636 non-null   object 
 3   AirTankers            28 non-null     float64
 4   ArchiveYear           1636 non-null   int64  
 5   CalFireIncident       1636 non-null   bool   
 6   CanonicalUrl          1636 non-null   object 
 7   ConditionStatement    284 non-null    object 
 8   ControlStatement      111 non-null    object 
 9   Counties              1636 non-null   object 
 10  CountyIds             1636 non-null   object 
 11  CrewsInvolved         171 non-null    float64
 12  Dozers                123 non-null    float64
 13  Engines               191 non-null    float64
 14  Extinguished          1577 non-null   object 
 15  Fatalities           

In [17]:
cols=["Location", "SearchDescription", "SearchKeywords", 
"CanonicalUrl", "Final", "Featured", "FuelType", "Updated", 
"StructuresEvacuated", "Active", "ConditionStatement", 
"ControlStatement", "Public", "PercentContained", "Status"]
ds_copy = ds.copy()

ds_copy = ds_copy.drop(cols, axis = 1)
ds_copy["Extinguished"] = pd.to_datetime(ds_copy["Extinguished"])
ds_copy["Started"] = pd.to_datetime(ds_copy["Started"])
ds_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1636 entries, 0 to 1635
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   AcresBurned           1633 non-null   float64            
 1   AdminUnit             1636 non-null   object             
 2   AirTankers            28 non-null     float64            
 3   ArchiveYear           1636 non-null   int64              
 4   CalFireIncident       1636 non-null   bool               
 5   Counties              1636 non-null   object             
 6   CountyIds             1636 non-null   object             
 7   CrewsInvolved         171 non-null    float64            
 8   Dozers                123 non-null    float64            
 9   Engines               191 non-null    float64            
 10  Extinguished          1577 non-null   datetime64[ns, UTC]
 11  Fatalities            21 non-null     float64            
 12  Helico

In [18]:
ds_copy["FireDuration"] = -(ds_copy["Started"]-ds_copy["Extinguished"])
ds_copy.head()

,AcresBurned,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,Counties,CountyIds,CrewsInvolved,Dozers,Engines,...,MajorIncident,Name,PersonnelInvolved,Started,StructuresDamaged,StructuresDestroyed,StructuresThreatened,UniqueId,WaterTenders,FireDuration
0,257314.0,Stanislaus National Forest/Yosemite National Park,NaN,2013,True,Tuolumne,55,NaN,NaN,NaN,...,False,Rim Fire,NaN,2013-08-17 15:25:00+00:00,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,NaN,20 days 03:05:00
1,30274.0,USFS Angeles National Forest/Los Angeles Count...,NaN,2013,True,Los Angeles,19,NaN,NaN,NaN,...,False,Powerhouse Fire,NaN,2013-05-30 15:28:00+00:00,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,NaN,9 days 03:02:00
2,27531.0,CAL FIRE Riverside Unit / San Bernardino Natio...,NaN,2013,True,Riverside,33,NaN,NaN,NaN,...,False,Mountain Fire,NaN,2013-07-15 13:43:00+00:00,NaN,NaN,NaN,a3149fec-4d48-427c-8b2c-59e8b79d59db,NaN,15 days 04:17:00
3,27440.0,Tahoe National Forest,NaN,2013,False,Placer,31,NaN,NaN,NaN,...,False,American Fire,NaN,2013-08-10 16:30:00+00:00,NaN,NaN,NaN,8213f5c7-34fa-403b-a4bc-da2ace6e6625,NaN,19 days 15:30:00
4,24251.0,Ventura County Fire/CAL FIRE,NaN,2013,True,Ventura,56,47.0,8.0,117.0,...,True,Springs Fire,2167.0,2013-05-02 07:01:00+00:00,6.0,10.0,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,11.0,8 days 23:29:00


In [31]:
# How many different counties are there in the dataset
print(ds_copy["Counties"].value_counts)

<bound method IndexOpsMixin.value_counts of 0          Tuolumne
1       Los Angeles
2         Riverside
3            Placer
4           Ventura
           ...     
1631      Riverside
1632         Nevada
1633           Yolo
1634      San Diego
1635      Riverside
Name: Counties, Length: 1636, dtype: object>
